<h1><center>MLB In-Depth EDA</center></h1>
<h2><center>One Stop for all your needs!</center></h2>
                                                      
<center><img src = "https://upload.wikimedia.org/wikipedia/en/thumb/a/a6/Major_League_Baseball_logo.svg/1200px-Major_League_Baseball_logo.svg.png" width = "750" height = "500"/></center>                                                                                               

### **You can connect with me on [LinkedIn](https://www.linkedin.com/in/ishandutta0098)**

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Contents</center></h2>

1. [Competition Overview](#competition-overview)  
2. [Libraries](#libraries)  
3. [Global Config](#global-config)
4. [Weights and Biases](#weights-and-biases)
5. [Load Datasets](#load-datasets)  
6. [Awards Exploration](#awards-exploration)  
7. [Model](#model)

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='background:purple; border:0; color:white' role="tab" aria-controls="home"><center>If you find this notebook useful, do give me an upvote, it helps to keep up my motivation. This notebook will be updated frequently so keep checking for furthur developments.</center></h3>

<a id="competition-overview"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Competition Overview</center></h2>

Let's begin by understanding a brief overview of the competition. I like to code more than to write bulky paragraphs, so we will move straight ahead to the code part in the next step!

## Description
- In this competition, you’ll predict how fans engage with MLB players’ digital content on a daily basis for a future date range. 
- You’ll have access to player performance data, social media data, and team factors like market size. Successful models will provide new insights into what signals most strongly correlate with and influence engagement.

## Evaluation Criteria
- Submissions are evaluated on the mean column-wise mean absolute error (MCMAE). 
- A mean absolute error is calculated for each of the four target variables and the score is the average of those four MAE values.

<a id="libraries"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Libraries</center></h2>

<center><img src = "https://i.guim.co.uk/img/media/b53e226d3c70e90ddc25b877be945176b0470cc0/0_187_5616_3370/master/5616.jpg?width=1200&height=900&quality=85&auto=format&fit=crop&s=c77f3a62ff97312e0c9e8520ce23508a" width = "750" height = "500"/></center>                                                                                               

We will import all the necessary modules at one place to make our life easier :-)

In [ ]:
!pip install -q --upgrade wandb

In [ ]:
import numpy as np
import pandas as pd

import os

import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from wordcloud import WordCloud, STOPWORDS

#Text Color
from termcolor import colored

#Data Preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

#NLP
from sklearn.feature_extraction.text import CountVectorizer

#WordCloud
from wordcloud import WordCloud, STOPWORDS

#Text Processing
import re
import nltk
nltk.download('popular')

#Language Detection
!pip install langdetect
import langdetect

#Sentiment
from textblob import TextBlob

#ner
import spacy

#Vectorizer
from sklearn import feature_extraction, manifold

#Word Embedding
import gensim.downloader as gensim_api

#Topic Modeling
import gensim

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

import random

import wandb

from tqdm import tqdm
import os
from sklearn.model_selection import StratifiedKFold, TimeSeriesSplit

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Login to wandb
wandb.login()

<a id="global-config"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Global Config</center></h2>

In [ ]:
target_stats = pd.read_csv('../input/player-target-stats/player_target_stats.csv')

class CFG:
    nfolds = 5
    batch_size = 1024
    val_batch_size = batch_size * 8
    nepochs = 3
    lr = 0.001
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    seed = 42
    targets = ['target1', 'target2', 'target3', 'target4']
    engineered_cols = ['player_t1_mean', 'player_t2_mean', 'player_t3_mean', 'player_t4_mean']
    cols = ['month', 'week', 'weekday'] + engineered_cols + [col for col in target_stats.columns if col not in ['playerId']]
    debug = False

In [ ]:
# wandb config
WANDB_CONFIG = {
     'competition': 'MLB', 
              '_wandb_kernel': 'neuracort'
    }

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
set_seed()

<a id="weights-and-biases"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Weights and Biases</center></h2>

<center><img src = "https://i.imgur.com/1sm6x8P.png" width = "750" height = "500"/></center>        

**Weights & Biases** is the machine learning platform for developers to build better models faster.

You can use W&B's lightweight, interoperable tools to

- quickly track experiments,
- version and iterate on datasets,
- evaluate model performance,
- reproduce models,
- visualize results and spot regressions,
- and share findings with colleagues.
  
Set up W&B in 5 minutes, then quickly iterate on your machine learning pipeline with the confidence that your datasets and models are tracked and versioned in a reliable system of record.

In this notebook I will use Weights and Biases's amazing features to perform wonderful visualizations and logging seamlessly.

<a id="load-datasets"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Load Datasets</center></h2>

Now that we have imported all we need let's begin by using the toughest Python function `read_csv` ;-)

In [ ]:
awards = pd.read_csv("../input/c/mlb-player-digital-engagement-forecasting/awards.csv")
train = pd.read_csv('../input/unnest-train/train_nextDayPlayerEngagement.csv')

if CFG.debug:
    train = train[:10000]

Next we will do exploration on each of the files separately and as mentioned "InDepth".

<a id="awards-exploration"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Awards Exploration</center></h2>

<center><img src = "https://olc-wordpress-assets.s3.amazonaws.com/uploads/2021/04/OLC-Awards-Thumbnail-1200x800.jpg" width = "750" height = "500"/></center>                                                                                               

This file has awards won by players in the training set prior to the beginning of the daily data (i.e. before 2018).

- `awardDate` - Date award was given.
- `awardSeason` - Season award was from.
- `awardId`
- `awardName`
- `playerId` - Unique identifier for a player.
- `playerName`
- `awardPlayerTeamId`

Let's take a quick peek into the dataset.

In [ ]:
awards.head()

## **<span style="color:orange;">Dataset Size</span>** 

In [ ]:
print(f"Dataset Shape: {colored(awards.shape, 'yellow')}")

## **<span style="color:orange;">Dataset Info</span>** 

Now we shall check if there are any null values in the dataset and if the datatypes are correct.

In [ ]:
awards.info()

**OBSERVATIONS**
- `awardDate`: No Null Values but data type is incorrect it should be `datetime`
- `awardSeason`: No Null Values and data type is correct.
- `awardId`: No Null Values and data type is correct.
- `awardName`: No Null Values and data type is correct.
- `playerId`: No Null Values and data type is correct.
- `playerName`: No Null Values and data type is correct.
- `awardPlayerTeamId`: 13 Null Values and data type is correct.

Since only 0.1% of the data has Null Values we can drop those rows and analyse with the rest of the dataset.

## **<span style="color:orange;">Cleaning Dataset</span>** 

In [ ]:
# Dropping Null Values
awards.dropna(inplace = True)

# Correcting DataType of `awardDate`
awards["awardDate"] = pd.to_datetime(awards["awardDate"])

Continuing further, we would look for unique values in the data to gain an intuition about repeated values.

In [ ]:
for col in awards.columns:
    print(col + ":" + colored(str(len(awards[col].unique())), 'yellow'))

We can infer that all columns have repeated values.

## **<span style="color:orange;">Text Column Preprocessing</span>** 

We shall preprocess the Text Columns to analyse them in a better way.

In [ ]:
def preprocess_text(text, flg_stemm=False, flg_lemm=True):

    lst_stopwords = nltk.corpus.stopwords.words("english")
    
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()    
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

def apply_preprocess_text(df, column):

    #Clean text
    clean_column = "clean_" + column
    df[clean_column] = df[column].apply(lambda x: preprocess_text(x, flg_stemm=False, flg_lemm=True, ))

    #Length of text
    clean_column_len = "clean_" + column + "_len"
    df[clean_column_len] = df[clean_column].apply(lambda x: len(x))

    #Word Count
    clean_column_word_count = "clean_" + column + "_word_count"
    df[clean_column_word_count] =df[clean_column].apply(lambda x: len(str(x).split(" ")))

    #Character Count
    clean_column_char_count = "clean_" + column + "_char_count"
    df[clean_column_char_count] = df[clean_column].apply(lambda x: sum(len(word) for word in str(x).split(" ")))

    #Average Word Length
    clean_column_avg_word_length = "clean_" + column + "_avg_word_length"
    df[clean_column_avg_word_length] = df[clean_column_char_count] / df[clean_column_word_count]

In [ ]:
apply_preprocess_text(awards, "awardName")
apply_preprocess_text(awards, "playerName")

## **<span style="color:orange;">Distribution Plot</span>** 

In [ ]:
import plotly.express as px
def plot_distribution(df, x, title):

    fig = px.histogram(
    df, 
    x = x,
    width = 800,
    height = 500,
    title = title
    )
    
    fig.show()

### 1. Award Name Distributions

In [ ]:
plot_distribution(df = awards, x = 'clean_awardName_len', title = 'Award Name Length Distribution')

In [ ]:
plot_distribution(df = awards, x = 'clean_awardName_word_count', title = 'Word Count Distribution')

In [ ]:
plot_distribution(df = awards, x = 'clean_awardName_char_count', title = 'Character Count Distribution')

In [ ]:
plot_distribution(df = awards, x = 'clean_awardName_avg_word_length', title = 'Average Word Length Distribution')

### 2. Player Name Distributions

In [ ]:
plot_distribution(df = awards, x = 'clean_playerName_len', title = 'Award Name Length Distribution')

In [ ]:
plot_distribution(df = awards, x = 'clean_playerName_word_count', title = 'Word Count Distribution')

In [ ]:
plot_distribution(df = awards, x = 'clean_playerName_char_count', title = 'Character Count Distribution')

In [ ]:
plot_distribution(df = awards, x = 'clean_playerName_char_count', title = 'Character Count Distribution')

<a id="model"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Model</center></h2>

In [ ]:
class MLBDataset(Dataset):
    def __init__(self, df, targets=None, mode='train'):
        self.mode = mode
        self.data = df
        if mode == 'train':
            self.targets = targets
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if self.mode == 'train':
            x = self.data[idx]
            y = np.array(self.targets[idx])
            return torch.from_numpy(x).float(), torch.from_numpy(y).float()
        elif self.mode == 'test':
            return torch.from_numpy(self.data[idx]).float()
        
class MLBModel(nn.Module):
    def __init__(self, num_cols):
        super(MLBModel, self).__init__()
        self.dense1 = nn.Linear(num_cols, 100)
        self.dense2 = nn.Linear(100, 100)
        self.dense3 = nn.Linear(100, len(CFG.targets))

    def forward(self, x):
        x = F.relu(self.dense1(x))
        x = F.relu(self.dense2(x))
        x = self.dense3(x).squeeze()

        return x

In [ ]:
def add_player_features(row, player_dict, n_feats, id_col, nan_value=-1):
    features = np.full((n_feats), nan_value, dtype=np.float32)
    
    pid = row[id_col]
    
    if pid in player_dict:
        # overall player means
        p_count = player_dict[pid]['count']
        features[0] = player_dict[pid]['mean_t1'] / p_count
        features[1] = player_dict[pid]['mean_t2'] / p_count
        features[2] = player_dict[pid]['mean_t3'] / p_count
        features[3] = player_dict[pid]['mean_t4'] / p_count
    
    return features

def update_player_features(row, player_dict, nan_value=-1):
    assert player_dict is not None
    
    pid = row[2]
    t1, t2, t3, t4 = row[3], row[4], row[5], row[6]
    
    if pid in player_dict:
        # update target lags
        player_dict[pid]['mean_t1'] += t1
        player_dict[pid]['mean_t2'] += t2
        player_dict[pid]['mean_t3'] += t3
        player_dict[pid]['mean_t4'] += t4
        player_dict[pid]['count'] += 1
    else:
        # init the feature dict for this player
        player_dict[pid] = {'mean_t1': t1, 'mean_t2': t2, 'mean_t3': t3, 'mean_t4': t4, 'count': 1}
    
    return player_dict

def do_feature_engineering(df, id_col=2):
    player_features = {}
    nfeats = len(CFG.engineered_cols)
    features = np.zeros((df.shape[0], nfeats), dtype=np.float32)
    
    for idx, row in enumerate(tqdm(df.values)):
        row_features = add_player_features(row, player_features, nfeats, id_col=id_col)
        player_features = update_player_features(row, player_features)
        features[idx] = row_features
    
    features = pd.DataFrame(features, columns=CFG.engineered_cols)
    df = pd.concat([df, features], axis=1)
    
    return df, player_features

def do_feature_engineering_test(df, player_dict, id_col=2):
    features = np.zeros((df.shape[0], len(CFG.engineered_cols)), dtype=np.float32)
    
    for idx, row in enumerate(df.values):
        row_features = add_player_features(row, player_dict, len(CFG.engineered_cols), id_col=id_col)
        features[idx] = row_features
    
    features = pd.DataFrame(features, columns=CFG.engineered_cols, index=df.index)
    df = pd.concat([df, features], axis=1)
    
    return df

def train_epoch(model, loader, optimizer, device, criterion):
    model.train()
    train_loss = 0.0

    for i, (sample, target) in enumerate(tqdm(loader)):
        sample, target = sample.to(device), target.to(device)
        optimizer.zero_grad()

        preds = model(sample)

        loss = criterion(preds, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() / len(loader)

    return model, train_loss

def validate(model, loader, device, criterion):
    model.eval()
    val_loss = 0.0
    val_preds = []
    val_targets = []

    with torch.no_grad():
        for i, (sample, target) in enumerate(tqdm(loader)):
            sample, target = sample.to(device), target.to(device)

            preds = model(sample)
            loss = criterion(preds, target)
            val_loss += loss.item() / len(loader)

            val_preds.append(preds.cpu())
            val_targets.append(target.cpu())

        val_preds = np.concatenate(val_preds)
        val_targets = np.concatenate(val_targets)

    return val_loss, val_preds

class ColL1Loss(nn.Module):
    def __init__(self, num_targets):
        super().__init__()
        self.mae = nn.L1Loss()
        assert num_targets != 0
        self.num_targets = num_targets
    
    def forward(self, preds, targets):
        l1 = 0.0
        
        for i in range(self.num_targets):
            l1 += self.mae(preds[:, i], targets[:, i])
        
        return l1 / self.num_targets
    
def train_fold(xtrn, ytrn, xval, yval, fold):
    print(f"Train fold {fold}")
    train_set = MLBDataset(xtrn, ytrn)
    val_set = MLBDataset(xval, yval)
    
    train_loader = DataLoader(train_set, batch_size=CFG.batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=CFG.val_batch_size, shuffle=False)
    
    model = MLBModel(xtrn.shape[1]).to(CFG.device)
    criterion = ColL1Loss(len(CFG.targets))
    optimizer = optim.Adam(model.parameters(), lr=CFG.lr)
    
    best_loss = {'train': np.inf, 'val': np.inf}
    best_val_preds = None
    
    # Initialize W&B
    run = wandb.init(project='MLB', config= WANDB_CONFIG)
    
    for epoch in range(1, CFG.nepochs + 1):
        print(f"Train epoch {epoch}")
        model, loss = train_epoch(model, train_loader, optimizer, CFG.device, criterion)
        val_loss, val_preds = validate(model, val_loader, CFG.device, criterion)
        
        # save best model
        if val_loss < best_loss['val']:
            best_loss = {'train': loss, 'val': val_loss}
            torch.save(model.state_dict(), f'fold{fold}.pt')
            
            # save best oof predictions
            best_val_preds = val_preds
            np.save(f'oof_fold{fold}.npy', val_preds)
        
        print("Train loss: {:5.5f}".format(loss))
        print("Val loss: {:5.5f}".format(val_loss))
        
        wandb.log({
            'train_loss': loss,
            'valid_loss': val_loss
        })
        
    # Close W&B run
    wandb.finish()
    
    return best_val_preds

def extract_date_feats(df):
    df['year'] = pd.DatetimeIndex(df['engagementMetricsDate']).year
    df['month'] = pd.DatetimeIndex(df['engagementMetricsDate']).month
    df['week'] = pd.DatetimeIndex(df['engagementMetricsDate']).week
    df['weekday'] = pd.DatetimeIndex(df['engagementMetricsDate']).weekday
    
    return df

def train_kfolds(df):
    set_seed(CFG.seed)
    ts = TimeSeriesSplit(n_splits=CFG.nfolds)
    df = extract_date_feats(df)
    
    oof_preds = []
    oof_targets = []
    
    for fold_idx, (trn_idx, val_idx) in enumerate(ts.split(df)):
        trn_df, val_df = df.iloc[trn_idx], df.iloc[val_idx]
        
        trn_df, player_dict = do_feature_engineering(trn_df)
        val_df = do_feature_engineering_test(val_df, player_dict)
        trn_df = pd.merge(trn_df, target_stats, how='left', on='playerId')
        val_df = pd.merge(val_df, target_stats, how='left', on='playerId')
        
        xtrn, ytrn = trn_df[CFG.cols].values, trn_df[CFG.targets].values
        xval, yval = val_df[CFG.cols].values, val_df[CFG.targets].values
        
        val_preds = train_fold(xtrn, ytrn, xval, yval, fold_idx)
        
        oof_preds.append(val_preds)
        oof_targets.append(yval)
        del trn_df, val_df, player_dict
        del xtrn, ytrn, xval, yval
    
    oof_preds = torch.from_numpy(np.clip(np.concatenate(oof_preds), 0, 100)).float()
    oof_targets = torch.from_numpy(np.concatenate(oof_targets)).float()
    
    eval_metric = ColL1Loss(len(CFG.targets))
    oof_loss = eval_metric(oof_preds, oof_targets)
    df, player_dict = do_feature_engineering(df)
    del oof_preds, oof_targets
    del df
    
    return oof_loss, player_dict

In [ ]:
loss, player_dict = train_kfolds(train)
print("OOF loss: {:5.5f}".format(loss))
del train

--- 

## **<span style="color:orange;">Let's have a Talk!</span>**
> ### Reach out to me on [LinkedIn](https://www.linkedin.com/in/ishandutta0098)

---